In [1]:
# Update sklearn to prevent version mismatches
# !pip install sklearn --upgrade

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1321 sha256=6e9b7149fefd52a6c32e91f3eba65cb8897243c946626b175d1cadf1097b7b4b
  Stored in directory: C:\Users\dbs2019\AppData\Local\pip\Cache\wheels\76\03\bb\589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
# !pip install joblib

In [7]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Read the CSV and Perform Basic Data Cleaning

In [8]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [9]:
# Remove Space for `FALSE POSITIVE` category
mask = df["koi_disposition"] == "FALSE POSITIVE"
df.loc[mask, "koi_disposition"] = "False_Positive"
df["koi_disposition"]

0            CONFIRMED
1       False_Positive
2       False_Positive
3            CONFIRMED
4            CONFIRMED
             ...      
6986    False_Positive
6987    False_Positive
6988         CANDIDATE
6989    False_Positive
6990    False_Positive
Name: koi_disposition, Length: 6991, dtype: object

# Create a Train Test Split

Use `koi_disposition` for the y values

In [10]:
X = df.drop("koi_disposition", axis=1)
y = df["koi_disposition"]
print(X.shape, y.shape)

(6991, 40) (6991,)


In [11]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [12]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
3563,0,0,0,0,10.548413,5.470000e-05,-5.470000e-05,139.064020,0.004110,-0.004110,...,-133,4.387,0.066,-0.123,1.092,0.181,-0.097,298.09543,44.737061,13.204
4099,0,0,0,0,24.754385,1.365000e-04,-1.365000e-04,140.207320,0.004460,-0.004460,...,-144,4.519,0.078,-0.052,0.804,0.056,-0.076,295.73535,42.576248,15.514
5460,0,0,0,0,1.057336,1.230000e-07,-1.230000e-07,131.792007,0.000096,-0.000096,...,-140,4.594,0.054,-0.027,0.683,0.054,-0.060,292.18417,49.310040,15.414
1091,0,0,0,0,201.118319,1.461000e-03,-1.461000e-03,187.569860,0.005290,-0.005290,...,-112,4.447,0.072,-0.108,0.954,0.135,-0.083,283.11377,48.131390,13.328
5999,0,0,0,0,91.649983,3.181000e-03,-3.181000e-03,175.715600,0.028600,-0.028600,...,-233,4.145,0.164,-0.164,1.608,0.905,-0.383,294.93198,39.812420,12.964


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [13]:
X_scaler = MinMaxScaler().fit(X_train)

In [14]:
# Transform the training and testing data using the X_scaler

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
print(f"Training Data Score: {model2.score(X_train_scaled, y_train)}")
print(f"Testing Data Score: {model2.score(X_test_scaled, y_test)}")

In [15]:
# Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

In [16]:
# One-hot encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)
y_test_categorical.shape

(1748, 3)

In [19]:
# Deep Learning
model = Sequential()
model.add(Dense(units=200, activation='relu', input_dim=40))
# model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=20, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [20]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 200)               8200      
_________________________________________________________________
dense_5 (Dense)              (None, 20)                4020      
_________________________________________________________________
dense_6 (Dense)              (None, 3)                 63        
Total params: 12,283
Trainable params: 12,283
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.compile(optimizer = 'adam',
             loss = 'categorical_crossentropy',
             metrics = ['accuracy'])

In [23]:
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=150,
    shuffle=True,
    verbose=2
)

Train on 5243 samples
Epoch 1/150
5243/5243 - 0s - loss: 0.5723 - accuracy: 0.7299
Epoch 2/150
5243/5243 - 0s - loss: 0.3797 - accuracy: 0.8058
Epoch 3/150
5243/5243 - 0s - loss: 0.3620 - accuracy: 0.8159
Epoch 4/150
5243/5243 - 0s - loss: 0.3515 - accuracy: 0.8274
Epoch 5/150
5243/5243 - 0s - loss: 0.3515 - accuracy: 0.8220
Epoch 6/150
5243/5243 - 0s - loss: 0.3379 - accuracy: 0.8350
Epoch 7/150
5243/5243 - 0s - loss: 0.3353 - accuracy: 0.8405
Epoch 8/150
5243/5243 - 0s - loss: 0.3316 - accuracy: 0.8402
Epoch 9/150
5243/5243 - 0s - loss: 0.3286 - accuracy: 0.8478
Epoch 10/150
5243/5243 - 0s - loss: 0.3235 - accuracy: 0.8489
Epoch 11/150
5243/5243 - 0s - loss: 0.3241 - accuracy: 0.8442
Epoch 12/150
5243/5243 - 0s - loss: 0.3152 - accuracy: 0.8571
Epoch 13/150
5243/5243 - 0s - loss: 0.3102 - accuracy: 0.8562
Epoch 14/150
5243/5243 - 0s - loss: 0.3130 - accuracy: 0.8575
Epoch 15/150
5243/5243 - 0s - loss: 0.3164 - accuracy: 0.8528
Epoch 16/150
5243/5243 - 0s - loss: 0.3102 - accuracy: 0.

Epoch 133/150
5243/5243 - 0s - loss: 0.2165 - accuracy: 0.9044
Epoch 134/150
5243/5243 - 0s - loss: 0.2157 - accuracy: 0.9039
Epoch 135/150
5243/5243 - 0s - loss: 0.2216 - accuracy: 0.9016
Epoch 136/150
5243/5243 - 0s - loss: 0.2153 - accuracy: 0.9052
Epoch 137/150
5243/5243 - 0s - loss: 0.2192 - accuracy: 0.9033
Epoch 138/150
5243/5243 - 0s - loss: 0.2189 - accuracy: 0.9020
Epoch 139/150
5243/5243 - 0s - loss: 0.2151 - accuracy: 0.9064
Epoch 140/150
5243/5243 - 0s - loss: 0.2154 - accuracy: 0.9056
Epoch 141/150
5243/5243 - 0s - loss: 0.2189 - accuracy: 0.9035
Epoch 142/150
5243/5243 - 0s - loss: 0.2162 - accuracy: 0.9033
Epoch 143/150
5243/5243 - 0s - loss: 0.2172 - accuracy: 0.9054
Epoch 144/150
5243/5243 - 0s - loss: 0.2212 - accuracy: 0.9033
Epoch 145/150
5243/5243 - 0s - loss: 0.2157 - accuracy: 0.9020
Epoch 146/150
5243/5243 - 0s - loss: 0.2201 - accuracy: 0.9025
Epoch 147/150
5243/5243 - 0s - loss: 0.2125 - accuracy: 0.9029
Epoch 148/150
5243/5243 - 0s - loss: 0.2106 - accuracy:

In [24]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

1748/1748 - 0s - loss: 0.2501 - accuracy: 0.9033
Deep Neural Network - Loss: 0.2501142224130696, Accuracy: 0.9033181071281433


In [25]:
predictions_encoded = model.predict_classes(X_test_scaled[:10])
predictions_labels = label_encoder.inverse_transform(predictions_encoded)

In [26]:
print("Encoded Predictions: " , predictions_encoded)
print("Prediction Labels: ", predictions_labels)
print("Actual Labels: ", list(y_test[:10]))

Encoded Predictions:  [1 2 2 1 2 2 2 0 1 2]
Prediction Labels:  ['CONFIRMED' 'False_Positive' 'False_Positive' 'CONFIRMED'
 'False_Positive' 'False_Positive' 'False_Positive' 'CANDIDATE'
 'CONFIRMED' 'False_Positive']
Actual Labels:  ['CONFIRMED', 'False_Positive', 'False_Positive', 'CONFIRMED', 'False_Positive', 'False_Positive', 'False_Positive', 'CONFIRMED', 'CANDIDATE', 'False_Positive']
